# Day 4: Wrapper & Embedded Selection

## Learning Objectives
- Implement Recursive Feature Elimination (RFE)
- Use tree-based feature importance
- L1 regularization for feature selection

---

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("✅ Libraries loaded!")
print("📚 Day 4: Wrapper & Embedded Selection")

✅ Libraries loaded!
📚 Day 4: Wrapper & Embedded Selection


In [4]:
# ============================================================
# BUILD FEATURE SET
# ============================================================

print("BUILDING FEATURE SET")
print("="*60)

np.random.seed(42)
n_days = 1500

returns = np.random.normal(0.0002, 0.015, n_days)
for i in range(1, len(returns)):
    returns[i] += 0.05 * returns[i-1]

prices = 100 * np.cumprod(1 + returns)
df = pd.DataFrame({'price': prices, 'returns': returns})

# Create features
for h in [1, 5, 10, 20, 60]:
    df[f'ret_{h}d'] = df['price'].pct_change(h)
    df[f'vol_{h}d'] = df['returns'].rolling(h).std()
    df[f'mom_{h}d'] = df['returns'].rolling(h).sum()

# Technical
gains = df['returns'].where(df['returns'] > 0, 0).rolling(14).mean()
losses = (-df['returns'].where(df['returns'] < 0, 0)).rolling(14).mean()
df['rsi'] = 100 - (100 / (1 + gains / (losses + 1e-10)))

df['ma_ratio'] = df['price'].rolling(5).mean() / df['price'].rolling(20).mean()
df['vol_ratio'] = df['vol_5d'] / df['vol_20d']

# Target
df['target'] = (df['returns'].shift(-1) > 0).astype(int)
df = df.dropna()

feature_cols = [c for c in df.columns if c not in ['price', 'returns', 'target']]
X = df[feature_cols].values
y = df['target'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

split = int(len(X) * 0.7)
X_train, X_test = X_scaled[:split], X_scaled[split:]
y_train, y_test = y[:split], y[split:]

print(f"Features: {len(feature_cols)}")
print(f"Train: {len(X_train)}, Test: {len(X_test)}")

BUILDING FEATURE SET


ValueError: Found array with 0 sample(s) (shape=(0, 18)) while a minimum of 1 is required by StandardScaler.

## Part 1: Recursive Feature Elimination (RFE)

In [ ]:
# ============================================================
# RECURSIVE FEATURE ELIMINATION
# ============================================================

print("RECURSIVE FEATURE ELIMINATION")
print("="*60)

# RFE with Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rfe = RFE(rf, n_features_to_select=8, step=1)
rfe.fit(X_train, y_train)

rfe_ranking = pd.DataFrame({
    'feature': feature_cols,
    'rank': rfe.ranking_,
    'selected': rfe.support_
}).sort_values('rank')

print("\nRFE Ranking:")
print(rfe_ranking.to_string(index=False))

# Selected features
rfe_features = [f for f, s in zip(feature_cols, rfe.support_) if s]
print(f"\nSelected: {rfe_features}")

In [ ]:
# ============================================================
# RFE WITH CROSS-VALIDATION
# ============================================================

print("\nRFECV: FINDING OPTIMAL NUMBER OF FEATURES")
print("="*60)

rfecv = RFECV(rf, step=1, cv=5, scoring='accuracy', min_features_to_select=3)
rfecv.fit(X_train, y_train)

print(f"Optimal number of features: {rfecv.n_features_}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(range(1, len(rfecv.cv_results_['mean_test_score']) + 1), 
        rfecv.cv_results_['mean_test_score'], marker='o')
ax.axvline(rfecv.n_features_, color='red', linestyle='--', label=f'Optimal: {rfecv.n_features_}')
ax.set_xlabel('Number of Features')
ax.set_ylabel('CV Accuracy')
ax.set_title('RFECV: Feature Selection Performance', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Part 2: Tree-Based Feature Importance

In [ ]:
# ============================================================
# RANDOM FOREST IMPORTANCE
# ============================================================

print("TREE-BASED IMPORTANCE")
print("="*60)

# Train full RF
rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Gini importance
gini_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nGini Importance:")
print(gini_importance.head(10).to_string(index=False))

# Permutation importance (more reliable)
perm_imp = permutation_importance(rf, X_test, y_test, n_repeats=10, random_state=42)

perm_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': perm_imp.importances_mean,
    'std': perm_imp.importances_std
}).sort_values('importance', ascending=False)

print("\nPermutation Importance:")
print(perm_importance.head(10).to_string(index=False))

In [ ]:
# ============================================================
# COMPARE IMPORTANCE METHODS
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

gini_importance.head(12).plot(x='feature', y='importance', kind='barh', ax=axes[0],
                               color='steelblue', alpha=0.7, legend=False)
axes[0].set_xlabel('Gini Importance')
axes[0].set_title('Random Forest: Gini Importance', fontweight='bold')
axes[0].invert_yaxis()

perm_importance.head(12).plot(x='feature', y='importance', kind='barh', ax=axes[1],
                               color='coral', alpha=0.7, legend=False)
axes[1].set_xlabel('Permutation Importance')
axes[1].set_title('Permutation Importance (on test set)', fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## Part 3: L1 Regularization (Lasso)

In [ ]:
# ============================================================
# L1 REGULARIZATION
# ============================================================

print("L1 REGULARIZATION (LASSO)")
print("="*60)

# Logistic Regression with L1
lr_l1 = LogisticRegression(penalty='l1', solver='saga', C=0.5, max_iter=1000, random_state=42)
lr_l1.fit(X_train, y_train)

l1_importance = pd.DataFrame({
    'feature': feature_cols,
    'coefficient': np.abs(lr_l1.coef_[0]),
    'sign': np.sign(lr_l1.coef_[0])
}).sort_values('coefficient', ascending=False)

print("\nL1 Coefficients:")
print(l1_importance.head(10).to_string(index=False))

# Non-zero features
l1_selected = l1_importance[l1_importance['coefficient'] > 0.001]['feature'].tolist()
print(f"\nNon-zero features ({len(l1_selected)}): {l1_selected}")

## Part 4: Consensus Selection

In [ ]:
# ============================================================
# CONSENSUS RANKING
# ============================================================

print("CONSENSUS RANKING")
print("="*60)

# Combine rankings
consensus = pd.DataFrame({'feature': feature_cols})

# Add ranks
consensus['rfe_rank'] = consensus['feature'].map(
    dict(zip(rfe_ranking['feature'], rfe_ranking['rank'])))
consensus['gini_rank'] = consensus['feature'].map(
    dict(zip(gini_importance['feature'], range(1, len(gini_importance)+1))))
consensus['perm_rank'] = consensus['feature'].map(
    dict(zip(perm_importance['feature'], range(1, len(perm_importance)+1))))
consensus['l1_rank'] = consensus['feature'].map(
    dict(zip(l1_importance['feature'], range(1, len(l1_importance)+1))))

consensus['avg_rank'] = consensus[['rfe_rank', 'gini_rank', 'perm_rank', 'l1_rank']].mean(axis=1)
consensus = consensus.sort_values('avg_rank')

print("\nConsensus Ranking:")
print(consensus.head(10).to_string(index=False))

final_features = consensus.head(8)['feature'].tolist()
print(f"\n✅ Final selected features: {final_features}")

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║        DAY 4 COMPLETE: WRAPPER & EMBEDDED SELECTION              ║
╠══════════════════════════════════════════════════════════════════╣
║  ✓ RFE and RFECV for optimal feature count                      ║
║  ✓ Random Forest Gini importance                                ║
║  ✓ Permutation importance (more reliable)                       ║
║  ✓ L1 regularization (Lasso) for sparse selection               ║
║  ✓ Consensus ranking across methods                             ║
╚══════════════════════════════════════════════════════════════════╝

Tomorrow: Day 5 - SHAP Analysis
""")